# Lab3 - Black Box EA

Wrote a local-search algorithm (eg. an EA) able to solve the Problem instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.


In [128]:
from lab3_lib import make_problem, AbstractProblem
from tqdm.autonotebook import tqdm, trange
from collections import namedtuple
from copy import deepcopy
from dataclasses import dataclass, field
import random
from typing import Literal, Union
import logging
import numpy as np

def clamp(n: Union[int, float], smallest: Union[int, float], largest: Union[int, float]) -> Union[int, float]: 
    return max(smallest, min(n, largest))

In [121]:
@dataclass(frozen=True)
class Individual:
    pick_one_probability: float = field(default_factory=lambda: random.random())
    len_picks: float = field(default_factory=lambda: random.random())

    _fitness: dict[str, float] = field(default_factory=lambda: dict())

    def mutate(it: "Individual", sigma: Union[float, tuple[float, float]] = 0.1) -> "Individual":
        try:
            sigma_one, sigma_two = sigma
        except TypeError:
            sigma_one = sigma_two = sigma
        pick_one_probability = clamp(it.pick_one_probability + random.gauss(0, sigma_one), 0, 1)
        len_picks = it.len_picks + random.gauss(0, sigma_two)
        return Individual(pick_one_probability, len_picks)
    
    def fitness(self: "Individual", problem: "AbstractProblem") -> float:
        x = problem.x
        if not x in self._fitness:
            self._fitness[x] = problem(self(x))
        
        return self._fitness[x]
        
    
    def __call__(self, n: int) -> list[Literal[0,1]]:
        return random.choices(
            [0, 1],
            [1-self.pick_one_probability, self.pick_one_probability], 
            k=max(int(n*self.len_picks), 1)
            )

In [130]:
EPOCHS = 10
MU = 10
LAMBDA = 10
MUT_RATE = (0.1, 0.1)
PROB_SIZE = 10

In [135]:
def train(*, variant: Literal["comma", "plus"] = "comma",
          problem_size: int = None, mu: int = None, lambda_: int = None, epochs: int = None,
            mutation_rate: tuple[float, float] = None, training_factor: float = 1.1):
    if epochs is None:
        epochs = EPOCHS
    if problem_size is None:
        problem_size = PROB_SIZE
    if lambda_ is None:
        lambda_ = LAMBDA
    if mu is None:
        mu = MU
    if mutation_rate is None:
        mutation_rate = deepcopy(MUT_RATE)

    problem = make_problem(problem_size)

    parents = [Individual() for _ in range(mu)]
    parents_result = [p.fitness(problem) for p in parents]
    pbar = trange(0, epochs, unit="epoch")
    streak_bar = tqdm(total=lambda_, desc="Evaluating offspring fitness", unit="streak", colour="gray", leave=False)
    for _ in pbar:
        pbar.set_description(f"Training - Fitness: {max(parents_result):.2%} p Calls: {problem.calls}")
        offspring = [(random.choice(parents)).mutate(mutation_rate) for _ in range(lambda_)]
        results = []
        streak_bar.reset(total=lambda_)
        for i in offspring:
            results.append(i.fitness(problem))
            streak_bar.update(1)

        incrate = (np.sum([res > sum(parents_result)/len(parents_result) for res in results])/lambda_)

        if incrate > 1/5:
            mutation_rate = (mutation_rate[0]*training_factor, mutation_rate[1]*training_factor)
        elif incrate < 1/5:
            mutation_rate = (mutation_rate[0]/training_factor, mutation_rate[1]/training_factor)

        
        population = list(zip(results, offspring))
        if variant == "plus":
            population.extend(list(zip(parents_result, parents)))
        population = sorted(population, key=lambda i:i[0], reverse=True)[:mu]

        parents = [it[1] for it in population]
        parents_result = [it[0] for it in population]
    streak_bar.close()
    best_ind = np.argmax(parents_result)

    return {
        "best": (parents_result[best_ind], parents[best_ind]),
        "parents": list(zip(parents_result, parents)),
        "mutation_rate": mutation_rate
    }


In [143]:
train(epochs=100, variant='plus', mu=25, lambda_=2)['best']

  0%|          | 0/100 [00:00<?, ?epoch/s]

Evaluating offspring fitness:   0%|          | 0/2 [00:00<?, ?streak/s]

(10.0,
 Individual(pick_one_probability=0.952580762684342, len_picks=0.03278187584556006, _fitness={10: 10.0}))